## Tooling - GSW standalone board

In [1]:
import Switch as sw

module ser02.py: list_ports(), open_port("COMx"), r(addr), w(addr, value), close_ports()
COM14 - USB Serial Port (COM14)
COM15 - USB Serial Port (COM15)
COM16 - USB Serial Port (COM16)
COM3 - Intel(R) Active Management Technology - SOL (COM3)
COM17 - USB Serial Port (COM17)
COM13 - USB Serial Port (COM13)
Please input the port name (e.g. COMx): com13
Open serial port at: com13
('r(0xf383) = ', '03c0')
('r(0xf384) = ', '003c')
('w(0xf383) = ', '03c3')
('w(0xf384) = ', '003f')


#### Check if microcode enabled
After microcode download, reg0x456 = 0x0008 (**the parser microcode is valid**), reg0x457 = 0x0201 (**bit1=0 PCE is enabled**).

Without microcode downloaded, reg0x456 = 0x0000, reg0x457 = 0x0001

0x456 bit 3
<img src="GSW_reg0x456_bit3.PNG">

In [ ]:
print (hex(sw.r(0xf383)))
print (hex(sw.r(0xf384)))
print (hex(sw.r(0x456)))
print (hex(sw.r(0x457)))

#### Download microcode
It takes 25sec.

In [3]:
sw.pyGSW_HW_Init()
print "finished."

finished.


#### Global Switch Macro Hardware Reset

Reset all hardware modules including the register settings. This reset
acts similar to the hardware reset and is cleared by HW after Reset is
executed.
reg 0xF400 = 0x0002

In [ ]:
sw.w(0xf400, 2)

#### Reconnect the serial port 
And enable the UART - MDIO bridge

In [ ]:
sw.rc()

#### Close the serial port

In [ ]:
sw.s.close_port()

#### Port summary

In [ ]:
for a in range(4):
    sw.Phy_sum(a)
    print (" ")

#### Set the counter type for port n

In [ ]:
sw.set_counter(0)

#### List of capabilities

In [4]:
a = 0
while sw.pyGSW_CapGet(a)==0:
    a = a+1

Number of Ethernet ports : 7

Number of virtual ports : 6

Pcket buffer size[in Bytes]: : 131072

Buffer Segment size: : 256

Number of queues: : 32

Number of traffic meters: : 16

Number of rate shapers: : 32

Number of VLAN groups: : 64

Number of FIDs: : 64

Number of MAC entries: : 2048

Number of multicast entries : 64

Number of PPPoE sessions: : 16

Number of STAG VLAN groups: : 64

The end of capability list.  : 0



#### The used APIs for storm control

sw.pyGSW_QoS_MeterCfgSet(en, mid, cbs, ebs, rate)

pyGSW_QoS_MeterPortAssign(mid, dir, piid, peid)  - dir:0 None, 1 Ingress, 2 Egress, 3 Both. 

pyGSW_QoS_StormCfgSet(mid, bc, mc, uc)

pyGSW_QoS_WredCfgSet(pf, tm, redmin, redmax, yellowmin, yellowmax, greenmin, greenmax)

In [ ]:
#50Mbps
#sw.pyGSW_QoS_MeterCfgSet(1, 0, 500, 0, 50*1024)#port 0 ingress
#sw.pyGSW_QoS_MeterPortAssign(0, 1, 0, 0)

In [ ]:
#sw.pyGSW_QoS_MeterCfgSet(1, 1, 500, 0, 50*1024)
#port 3 ingress
#sw.pyGSW_QoS_MeterPortAssign(0, 1, 3, 0)

In [ ]:
#50Mbps
sw.pyGSW_QoS_MeterCfgSet(1, 0, 500, 0, 50*1024)
sw.pyGSW_QoS_StormCfgSet(0, 1, 1, 1)
sw.pyGSW_QoS_WredCfgSet(0, 0, 0, 0, 0, 0, 0xffff, 0xffff)

#### 2 * EASY GPY211 back to back throughput test
Use 3 ports (port 0, 2, 3) of GWT140 as trunk to connect 2 * EASY GPY211. Configure:

**Board #0:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:05

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:04

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:01/03/05

**Board #1:** 

MAC_TableEntryAdd 0 0 1000000 0 1 0 00:22:A2:00:00:00

MAC_TableEntryAdd 0 2 1000000 0 1 0 00:22:A2:00:00:02

MAC_TableEntryAdd 0 3 1000000 0 1 0 00:22:A2:00:00:04

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:01

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:03

MAC_TableEntryAdd 0 4 1000000 0 1 0 00:22:A2:00:00:05

Traffic input from Port 4, the 2.5G PHY, des MAC are 00:22:A2:00:00:00/02/04

pyGSW_MAC_TableEntryAdd(nFId, nPortId, nAgeTimer, nSVLAN_Id, bStaticEntry, nTrafficClass, nMAC_0, nMAC_1, nMAC_2, nMAC_3, nMAC_4, nMAC_5):


In [ ]:
#Board #0
for a in range(1):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)

In [ ]:
#Board #1
for a in range(1):
    sw.pyGSW_MAC_TableEntryAdd(0, 0, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x00+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 1, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x02+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 2, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x04+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 3, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x06+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x01+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x03+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x05+a*8)
    sw.pyGSW_MAC_TableEntryAdd(0, 4, 1000000, 0, 1, 0, 0x0, 0x22, 0xa2, 0x00, 0x00, 0x07+a*8)

In [ ]:
sw.pyGSW_MAC_TableEntryRead(1)
for a in range(100):
    if sw.pyGSW_MAC_TableEntryRead(0) == 1:
        break

#### Play around

In [ ]:
print (hex(sw.mdior(0, 0x15)))
print (hex(sw.mdior(0, 3)))

In [ ]:
sw.pyGSW_PceRuleReadf(0, 0)